In [ ]:
import pandas as pd
pd.DataFrame({'a': [1,2,3]})

## sample function to extract data from kraken

In [ ]:
import slumber
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from decimal import Decimal as D
import logging
#logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
import copy

import numpy as np

In [ ]:
class Exchange:
  def __init__(self, url, commissions):
    self.url = url
    self.commissions = commissions
    self.api = None
    self.session = None

  def get_requests_session(self):
    if self.session is not None:
      return self.session

    s = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 404, 502, 503, 504 ])
    s.mount('https://', HTTPAdapter(max_retries=retries))
    self.session = s
    return s

  def get_slumber_api(self, append_slash=True):
    if self.api is not None:
      return self.api

    s = self.get_requests_session()
    self.api = slumber.API(self.url, session=s, append_slash=append_slash)
    return self.api

  def fetch(self, ticker):
    raise ValueError("Not implemented")

  def postprocess(self, ob1, ticker):
    raise ValueError("Not implemented")



class Kraken(Exchange):
  def __init__(self):
    super().__init__(
      url="https://api.kraken.com/0/public/",
      commissions = D('0.14')/100
    )

  def fetch(self, ticker):
    logging.debug('fetch kraken')

    api = self.get_slumber_api(append_slash=False)
    ob1 = api.Depth.get(pair=ticker, count=30)
    return ob1

  def postprocess(self, ob1, ticker):
    """
    ob1 - output of self.fetch(ticker)
    """
    logging.debug('postprocess kraken')
    
    if ob1['error']:
        raise ValueError("Kraken error: %s"%(ob1['error']))

    ob2 = copy.deepcopy(ob1)
    ob2 = list(ob2['result'].values())[0]
    for k1 in ['asks', 'bids']:
        ob2[k1] = pd.DataFrame(ob2[k1]).rename(columns={0:'price', 1:'amount', 2:'timestamp'})
        # convert to Decimal
        for k2 in ['price', 'amount']:
            ob2[k1][k2] = [D(x) for x in ob2[k1][k2].values]

    # commissions
    ob2['asks']['price pre-commission'] =  ob2['asks']['price']
    ob2['bids']['price pre-commission'] =  ob2['bids']['price']
    ob2['asks']['price'] = ob2['asks']['price'] * (1+self.commissions)
    ob2['bids']['price'] = ob2['bids']['price'] * (1-self.commissions)

    # set side
    ob2['asks']['side'] = 'ask'
    ob2['bids']['side'] = 'bid'

    ob3 = pd.concat([ob2['asks'], ob2['bids']], axis=0).reset_index(drop=True)
    ob3['source'] = 'kraken.com'
    ob3['ticker'] = ticker

    return ob3

In [ ]:
kraken = Kraken()
ticker = 'XBTUSD'
order_book = kraken.fetch(ticker)
order_book = kraken.postprocess(order_book, ticker)

In [ ]:
order_book

In [ ]:
min_ask = order_book[order_book['side']=='ask']['price'].min()
max_bid = order_book[order_book['side']=='bid']['price'].max()
spread = min_ask - max_bid
spread_array.append(spread)

In [ ]:
volatility_std = np.std(spread_array)
volatility_var = np.var(spread_array)
print(volatility_std, volatility_var)

In [ ]:
pd.Series([float(x) for x in spread_array]).std()

In [ ]:
df_spread = pd.DataFrame({'spread': [float(x) for x in spread_array]})
df_spread

In [ ]:
df_spread.rolling(window=5).std()

In [ ]:
df_spread.plot(style='.-')

In [ ]:
df_spread.rolling(window=5).std().plot(style='-.')